<a href="https://colab.research.google.com/github/Mayank-Bansal24/LAPLSTM-Models/blob/main/Allint1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import losses as lo_ss
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
import string
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn import model_selection
from sklearn.datasets import make_classification
from sklearn.utils import class_weight
from sklearn.metrics import classification_report , roc_auc_score
import keras 
import keras.backend as K
from google.colab import drive
from numpy import savetxt
import os

In [2]:
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [3]:
t=1

In [4]:
os.chdir('/content/gdrive/MyDrive/laplstm/allin/t'+str(t))

In [5]:
import os
github_raw_link="https://raw.githubusercontent.com/Aryansh085/LAP/main/858417d1-7d54-4115-a01b-fdda5e03ada3_testing_combined_rows4035_disc_1_0p9_MULTIPLY_preproces_155cols.csv"
df1=pd.read_csv((github_raw_link),header = None)
dftemp=np.array(df1)
dftemp.shape
dftemp2=dftemp[:,5:]
dftemp2=np.array(dftemp2)
print(dftemp2.shape)

(4035, 150)


In [6]:
# dftemp2
dftemp2.shape
# dftemp2=pd.DataFrame(dftemp2)
# dftemp2.head()

(4035, 150)

--> muliply the reward of that instance into the loss of that instance to give weight

In [7]:
from sklearn.preprocessing import StandardScaler

slr=StandardScaler()
slr.fit(dftemp2)
dftemp2 = slr.transform(dftemp2)

In [8]:
# f1-score function
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val
    

In [9]:
y=np.array(dftemp[:,0])
print(y)
y_new=y[t:]
y_new.shape


[2. 0. 0. ... 0. 2. 2.]


(4034,)

In [10]:
x=np.array(dftemp2)
# print(x)
# print(x.shape)
x.shape,y_new.shape

((4035, 150), (4034,))

In [11]:
x_3d=[]
for i in range(4035-t):
  tmp=[]
  for j in range(t):
    tmp.append(x[j+i])
  x_3d.append(tmp)
x_3d= np.array(x_3d)
x_3d.shape

(4034, 1, 150)

In [12]:
train_test_ratio=0.8
print(x_3d.shape[0])
x_train=[]
y_train=[]
x_test=[]
y_test=[]
for i in range(x_3d.shape[0]):
  l=[]
  ly=[]
  no=np.random.randint(100)
  if(no<80):
    l=x_3d[i]
    ly=y_new[i]
    x_train.append(l)
    y_train.append(ly)
  else:
    l=x_3d[i]
    ly=y_new[i]
    x_test.append(l)
    y_test.append(ly)


x_train=np.array(x_train)
y_train=np.array(y_train)
x_test=np.array(x_test)
y_test=np.array(y_test)
x_train.shape,y_train.shape,x_test.shape,y_test.shape

4034


((3195, 1, 150), (3195,), (839, 1, 150), (839,))

In [13]:
cnt0=0
cnt1=0
cnt2=0
for i in range(x_train.shape[0]):
  if(y_train[i]==0):
    cnt0+=1
  elif(y_train[i]==1):
    cnt1+=1
  elif(y_train[i]==2):
    cnt2+=1
print(cnt0)
print(cnt1)
print(cnt2)  

1457
823
915


In [14]:
x_train_1=[]
y_train_1=[]
c1=0
c2=0
c3=0
x1=(cnt1-cnt0)/(cnt1)
x1=x1*(-100)
x1=int(x1)
print(x1)
x2=(cnt2-cnt0)/(cnt2)
x2=x2*(-100)
x2=int(x2)
print(x2)
for i in range(x_train.shape[0]):
  if(y_train[i]==0):
    x_train_1.append(x_train[i])
    y_train_1.append(0)
    c1+=1
  elif(y_train[i]==1):
    x_train_1.append(x_train[i])
    y_train_1.append(1)
    c2+=1
  elif(y_train[i]==2 ):
    x_train_1.append(x_train[i])
    y_train_1.append(2)
    c3=c3+1
for i in range(x_train.shape[0]):
  if(y_train[i]==1):
    no=np.random.randint(100)
    if(no>=100-x1):
      x_train_1.append(x_train[i])
      y_train_1.append(1)
      c2+=1
  elif(y_train[i]==2 ):
    no=np.random.randint(100)
    if(no>=100-x2):
      x_train_1.append(x_train[i])
      y_train_1.append(2)
      c3+=1

y_train=[]
y_train=y_train_1
x_train=[]
x_train=x_train_1
x_train=np.array(x_train)
y_train=np.array(y_train)
print(c1)
print(c2)
print(c3)

77
59
1457
1452
1451


In [15]:
sklearn_weights=class_weight.compute_class_weight(class_weight='balanced',classes=np.unique(y_train),y=y_train)
sklearn_weights=dict(zip(np.unique(y_train),sklearn_weights))
sklearn_weights

{0: 0.9974834134065431, 1: 1.0009182736455464, 2: 1.0016080863772112}

In [16]:

# model = Sequential()
# model.add(LSTM(200, activation='relu', return_sequences=True, input_shape=( t,150,)))
# model.add(LSTM(100, activation='relu', return_sequences=True))
# model.add(LSTM(50, activation='relu', return_sequences=True))
# model.add(LSTM(25, activation='relu'))
# model.add(Dense(20, activation='relu'))
# model.add(Dense(10, activation='relu'))
# model.add(Dense(3))
# model.compile(optimizer='SGD', loss='sparse_categorical_crossentropy',metrics=['accuracy'])
# history = model.fit(x_train, y_train, epochs=10, validation_split=0.2, verbose=1)
# test_output = model.predict(x_test, verbose=1)
# # history = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1)
# # test_output = model.predict(x=, verbose=0)
# # print(test_output)

In [17]:
# sklearn_weights={0:1738,1:2348,2:2362}
model = Sequential()
model.add(LSTM(200, activation='relu', return_sequences=True, input_shape=( t,150,)))
model.add(LSTM(400, activation='relu', return_sequences=True))
model.add(LSTM(800, activation='relu', return_sequences=True))
model.add(LSTM(150, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(3,activation='sigmoid'))
optimi=keras.optimizers.Adam()
model.compile(optimizer=optimi, loss='sparse_categorical_crossentropy',metrics=['accuracy',get_f1])

# history = model.fit(x_train, y_train, epochs=30, validation_split=0.2, verbose=1)
history = model.fit(x_train, y_train, epochs=25, validation_split=0.2, verbose=1,class_weight=sklearn_weights)
test_output = model.predict(x_test, verbose=1)


Epoch 1/25
109/109 [==============================] - 7s 16ms/step - loss: 1.0897 - accuracy: 0.4080 - get_f1: 1.1359 - val_loss: 1.2037 - val_accuracy: 0.0000e+00 - val_get_f1: 1.6592
Epoch 2/25
109/109 [==============================] - 1s 10ms/step - loss: 1.0773 - accuracy: 0.4177 - get_f1: 1.1017 - val_loss: 1.2052 - val_accuracy: 0.0000e+00 - val_get_f1: 1.6112
Epoch 3/25
109/109 [==============================] - 1s 11ms/step - loss: 1.0348 - accuracy: 0.4300 - get_f1: 1.1453 - val_loss: 1.1568 - val_accuracy: 0.2064 - val_get_f1: 1.8446
Epoch 4/25
109/109 [==============================] - 1s 10ms/step - loss: 0.9313 - accuracy: 0.5241 - get_f1: 1.0531 - val_loss: 1.0308 - val_accuracy: 0.3567 - val_get_f1: 1.6212
Epoch 5/25
109/109 [==============================] - 1s 11ms/step - loss: 0.7801 - accuracy: 0.5877 - get_f1: 0.9426 - val_loss: 0.8678 - val_accuracy: 0.4404 - val_get_f1: 1.4171
Epoch 6/25
109/109 [==============================] - 1s 11ms/step - loss: 0.6237 - acc

In [18]:
test_output_1=np.array([np.argmax(i)  for i in test_output])
from sklearn.metrics import confusion_matrix,accuracy_score
print(confusion_matrix(y_test,test_output_1))
test_output=pd.DataFrame(test_output)
print(accuracy_score(y_test,test_output_1))


[[161  75 134]
 [105  42  85]
 [107  38  92]]
0.35160905840286055


In [19]:
print(get_f1(y_test.astype('float32'), test_output_1.astype('float32')))

tf.Tensor(0.54973257, shape=(), dtype=float32)


In [20]:
x_train_reshaped = x_train.reshape(x_train.shape[0], -1)
x_test_reshaped = x_test.reshape(x_test.shape[0], -1)
savetxt('arrxtrain.csv', x_train_reshaped, delimiter=',')
savetxt('arrytrain.csv', y_train, delimiter=',')
savetxt('arrxtest.csv', x_test_reshaped, delimiter=',')
savetxt('arrytest.csv', y_test, delimiter=',')